In [2]:
import os
import tensorflow as tf
from tensorflow import keras
import librosa 
import numpy as np
import keras
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, add, Dense, Dropout
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.layers import Conv2D, MaxPool2D,Conv1D,MaxPooling1D,Reshape
from tensorflow.keras.utils import Sequence
import random as rd

In [2]:
def load_song(path):
    y, rate = librosa.load(path)
    data = np.abs(librosa.stft(y, n_fft=2048, hop_length=512))
    data = (data - data.mean()) / data.std()
    col = data.shape[1] 
    for i in range(1290,data.shape[1]):
        data = np.delete(data,0,axis = 1)
    return data

In [3]:
data_dir = 'Data/genres_original/'
genres = list(os.listdir(data_dir))
paths = []

for i in range(len(genres)):
    curr_path = data_dir + genres[i] + '/'
    titles = os.listdir(curr_path)
    for title in titles:
        paths.append((curr_path + title,i))


rd.shuffle(paths)


In [4]:
train_paths,test_paths = train_test_split(paths,test_size=0.3)
test_paths,val_paths = train_test_split(test_paths,test_size=0.3)

In [5]:
'''
model = Sequential()
model.add(Reshape((1025,1290,1),input_shape = (1025,1290)))
model.add(Conv2D(128,kernel_size = 3,activation = 'relu'))
model.add(MaxPool2D(pool_size =(3,3)))
model.add(Conv2D(64,kernel_size = 3,activation = 'relu'))
model.add(MaxPool2D(pool_size =(3,3)))
model.add(Dropout(0.25))
model.add(Conv2D(64,kernel_size = 3,activation = 'relu'))
model.add(MaxPool2D(pool_size =(3,3)))
model.add(Conv2D(64,kernel_size = 3,activation = 'relu'))
model.add(MaxPool2D(pool_size =(3,3)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(64,activation = 'relu'))
model.add(Dense(len(genres),activation = 'softmax'))
model.summary()'''
#10 % efficiency 

"\nmodel = Sequential()\nmodel.add(Reshape((1025,1290,1),input_shape = (1025,1290)))\nmodel.add(Conv2D(128,kernel_size = 3,activation = 'relu'))\nmodel.add(MaxPool2D(pool_size =(3,3)))\nmodel.add(Conv2D(64,kernel_size = 3,activation = 'relu'))\nmodel.add(MaxPool2D(pool_size =(3,3)))\nmodel.add(Dropout(0.25))\nmodel.add(Conv2D(64,kernel_size = 3,activation = 'relu'))\nmodel.add(MaxPool2D(pool_size =(3,3)))\nmodel.add(Conv2D(64,kernel_size = 3,activation = 'relu'))\nmodel.add(MaxPool2D(pool_size =(3,3)))\nmodel.add(Dropout(0.25))\nmodel.add(Flatten())\nmodel.add(Dense(64,activation = 'relu'))\nmodel.add(Dense(len(genres),activation = 'softmax'))\nmodel.summary()"

In [6]:
model = Sequential()
model.add(Reshape((1025, 1290,1),input_shape = (1025, 1290)))
model.add(Conv2D(128,kernel_size = (4,4),activation = 'relu'))
model.add(MaxPool2D(pool_size =(4,4)))
model.add(Dropout(0.25))
model.add(Conv2D(64,kernel_size = (4,4),activation = 'relu'))
model.add(MaxPool2D(pool_size =(4,4)))
model.add(Dropout(0.25))
model.add(Conv2D(64,kernel_size = (4,4),activation = 'relu'))
model.add(MaxPool2D(pool_size =(4,4)))
model.add(Dropout(0.25))
model.add(Conv2D(64,kernel_size = (4,4),activation = 'relu'))
model.add(MaxPool2D(pool_size =(4,4)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(32,activation = 'relu'))
model.add(Dense(len(genres),activation = 'softmax'))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 reshape (Reshape)           (None, 1025, 1290, 1)     0         
                                                                 
 conv2d (Conv2D)             (None, 1022, 1287, 128)   2176      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 255, 321, 128)    0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 255, 321, 128)     0         
                                                                 
 conv2d_1 (Conv2D)           (None, 252, 318, 64)      131136    
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 63, 79, 64)       0         
 2D)                                                    

In [7]:
base_model = tf.keras.applications.MobileNetV2(input_shape=(224,224,3),
                                               include_top=False,
                                               weights='imagenet')

In [8]:
early_stop  = EarlyStopping(monitor = 'val_loss',patience=4,verbose=1)
model.compile(optimizer='adam',
              loss=SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [9]:
class DataGenerator(Sequence):
    def __init__(self, file_set):
        self.paths = file_set

    def __len__(self):
        return int(len(self.paths))

    def __getitem__(self, idx):
        batch_x = np.array(load_song(self.paths[idx][0])).reshape((1,1025, 1290))
        batch_y = np.array(self.paths[idx][1]).reshape((1))
        return batch_x, batch_y

In [10]:
class DataGeneratorTo3dim(Sequence):
    def __init__(self, file_set):
        self.paths = file_set

    def __len__(self):
        return int(len(self.paths))

    def __getitem__(self, idx):
        batch_x = np.stack((np.resize(load_song(self.paths[idx][0])).reshape((1,1025, 1290),new_shape=(224,224)),np.zeros((224,224)),np.zeros((224,224))),axis=2)
        batch_y = np.array(self.paths[idx][1]).reshape((1))
        return batch_x, batch_y

In [11]:
train_gen = DataGenerator(train_paths)
val_gen = DataGenerator(val_paths)

In [12]:
train_gen3 = DataGenerator(train_paths)
a = train_gen3.__getitem__(0)
print(a[0].shape)


(1, 1025, 1290)


In [13]:
model.fit(train_gen, 
                    epochs=20,
                    verbose=1,
                    validation_data = val_gen,
                    callbacks = [early_stop]
                    )

Epoch 1/6


C:\Users\Filip\anaconda3\lib\site-packages\keras\backend.py:5585: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


699/699 [==============================] - 3367s 5s/step - loss: 2.2786 - accuracy: 0.1302 - val_loss: 2.1084 - val_accuracy: 0.1667
Epoch 2/6
699/699 [==============================] - 1613s 2s/step - loss: 2.1439 - accuracy: 0.1917 - val_loss: 2.0630 - val_accuracy: 0.2333
Epoch 3/6
699/699 [==============================] - 4847s 7s/step - loss: 1.8638 - accuracy: 0.3305 - val_loss: 1.9281 - val_accuracy: 0.2556
Epoch 4/6
699/699 [==============================] - 1401s 2s/step - loss: 1.7530 - accuracy: 0.3591 - val_loss: 1.6533 - val_accuracy: 0.3667
Epoch 5/6
699/699 [==============================] - 1388s 2s/step - loss: 1.6022 - accuracy: 0.4192 - val_loss: 1.8345 - val_accuracy: 0.3111
Epoch 6/6
699/699 [==============================] - 1379s 2s/step - loss: 1.5924 - accuracy: 0.4192 - val_loss: 1.5004 - val_accuracy: 0.4667


In [14]:
model.save("song_classifier_2")

INFO:tensorflow:Assets written to: song_classifier_1\assets


INFO:tensorflow:Assets written to: song_classifier_1\assets


In [15]:
test_gen = DataGenerator(test_paths)
results =model.evaluate(test_gen)

210/210 [==============================] - 77s 367ms/step - loss: 1.5126 - accuracy: 0.4381


In [16]:
gen = DataGenerator(paths)
score = 0
labels_score = [0 for i in range(len(genres))]
for i in range(len(paths)):
    a = model.predict(gen.__getitem__(i)[0],verbose = 0)
    if gen.__getitem__(i)[1][0] == np.argmax(a):
        labels_score[gen.__getitem__(i)[1][0]] += 1
        score += 1
print(score)
print(labels_score)

486
[36, 81, 54, 26, 79, 15, 66, 79, 11, 39]


In [21]:
print(model.predict_on_batch(test_gen.__getitem__(35)[0]))
print(test_paths[35][0])
print(test_gen.__getitem__(0)[1])

[[0.13485529 0.00586919 0.06989498 0.19992569 0.05677962 0.04619331
  0.07597676 0.13744643 0.02690804 0.24615061]]
Data/genres_original/rock/rock.00024.wav
[5]


FileNotFoundError: Unsuccessful TensorSliceReader constructor: Failed to find any matching files for song_classifier_2\variables\variables
 You may be trying to load on a different device from the computational device. Consider setting the `experimental_io_device` option in `tf.saved_model.LoadOptions` to the io_device such as '/job:localhost'.